In [1]:
# pip install -r requirements.txt -U -i https://pypi.douban.com/simple

In [6]:
# pip install protobuf==3.20.* -i https://pypi.douban.com/simple

In [8]:
# !PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION="python"

In [1]:
import warnings
warnings.filterwarnings("ignore")

## Load the Model

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/mbart-large-50-many-to-one-mmt")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/mbart-large-50-many-to-one-mmt")

In [3]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [4]:
sentence = "开空调的情况下，续航掉的太快了，特别是冬天天气冷的时候，不开空调不行，天气一冻，续航就掉的更快"

tokenizer.src_lang = 'zh_CN'
tokenizer.tgt_lang = 'en_XX'

encoded = tokenizer(sentence, return_tensors="pt").to(device)
model = model.to(device)
generated_tokens = model.generate(**encoded)
decoded = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)[0]

print(decoded)

If the air conditioner is on, the flight resumes too quickly, especially in winter when the weather is cold. If the air conditioner is not on, the flight resumes faster as soon as the weather is cold


## Deploy it

In [ ]:
config = dict()

config["S3_MODEL"] = "s3://hugging-face/llm/mbart.tar.gz"
config['MODEL_NAME'] = "mbart"

In [ ]:
import boto3  
from sagemaker.huggingface.model import HuggingFaceModel
import sagemaker
from time import gmtime, strftime

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

huggingface_model = HuggingFaceModel(
    model_data=config["S3_MODEL"],
    role=role,
    transformers_version='4.26',
    pytorch_version='1.13',
    py_version='py39',
)

endpoint_name = config['MODEL_NAME'].upper() + strftime("-%Y%m%d-%H%M%S", gmtime())

In [ ]:
predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type='ml.g4dn.4xlarge',
    endpoint_name=endpoint_name,
)

print(endpoint_name)

In [ ]:
from sagemaker.huggingface.model import HuggingFacePredictor

In [ ]:
predictor = HuggingFacePredictor(
    endpoint_name=endpoint_name
)